# Parameterselectie met ```groundhog```

In deze notebook wordt getoond hoe het Python package ```groundhog``` kan gebruikt worden om stratigrafische profielen te definiëren en parameters te selecteren.

## Import van bibliotheken

We zullen een aantal bibliotheken inladen die van pas komen tijdens de verwerking.

In [ ]:
import numpy as np
import os
from IPython.display import HTML
import pandas as pd
from copy import deepcopy

We zullen in eerste instantie Plotly gebruiken om grafieken te maken.

In [ ]:
from plotly import tools, subplots
import plotly.express as px
import plotly.graph_objs as go
import plotly.io as pio
import plotly.figure_factory as ff
from plotly.colors import DEFAULT_PLOTLY_COLORS
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode()
pio.templates.default = 'plotly_white'
pio.templates['plotly'].layout['autosize'] = False
for key in pio.templates.keys():
    pio.templates[key].layout['autosize'] = False

Uit ``groundhog`` laden we de functionaliteit voor CPT-verwerking, definitie van stratigrafische grondprofielen (klasse ``SoilProfile``) en inladen van startigrafische profielen (methode ``read_excel``) en plotten van startigrafische logs naast geotechnische data (``LogPlot`` en ``LogPlotMatplotlib``).

In [ ]:
from groundhog.siteinvestigation.insitutests.pcpt_processing import PCPTProcessing
from groundhog.general.soilprofile import SoilProfile, read_excel
from groundhog.general.plotting import LogPlot, LogPlotMatplotlib

## CPT data

We kunnen de data van CPT WFS1-1 uit de Borssele offshore wind farm inladen. De data is beschikbaar in de file ``CPT.xlsx`` in de subfolder ``Data``. We gebruiken de ``.load_excel`` methode om Excel files in te laden. De Excel file bevat de vereiste <i>headers</i>``z [m]``, ``qc [MPa]``, ``fs [MPa]`` en ``u2 [MPa]``.

In [ ]:
cpt = PCPTProcessing(title="Example CPT")
cpt.load_excel('Data/CPT.xlsx')
cpt.plot_raw_pcpt(u2_range=(-0.5, 2.5), u2_tick=0.5)

## Gelaagdheid

De gelaagdheid is gedefinieerd uit een nabijgelegen boring (file ``gelaagdheid.xlsx`` in de subfolder ``Data``) en kan geladen worden met de ``read_excel`` methode uit de ``groundhog.general.soilprofile`` module.

In [ ]:
gelaagdheid = read_excel('Data/gelaagdheid.xlsx')

We kunnen deze gelaagdheid weergeven:

In [ ]:
gelaagdheid

## Berekening van de wrijvingshoek met de correlatie van Kulhawy & Mayne (1990)

Zoals in de vorige notebook, kan de correlatie van Kulhawy & Mayne voor wrijvingshoeken ook op deze CPT worden toegepast. We berekenen eerst opnieuw $ q_t $, we zullen aannemen dat die gelijk is aan $ q_c $ (niet helemaal correct, zie cursus <i>Geotechnics</i> maar aanvaardbaar voor zanden en overgeconsolideerde kleien).

In [ ]:
cpt.data['qt [MPa]'] = cpt.data['qc [MPa]']

Voor berekening van de effectieve verticale spanning kunnen we de waarden in ``gelaagdheid`` gebruiken. De totale eenheidsgewichten worden naar het <i>grid</i> van de conuswaarden geïnterpoleerd en voor het effectieve eenheidsgewicht wordt gekeken of de laag boven of onder de watertafel ligt. In dit voorbeeld (offshore) ligt alles onder de watertafel.

In [ ]:
cpt.map_properties(layer_profile=gelaagdheid, waterlevel=0)

Als we de CPT data weergeven kunnen we zien dat de totale en effectieve vertical spanning berekend zijn uit de eenheidsgewichten gegeven in ``gelaagdheid``.

In [ ]:
cpt.data.head()

De correlatie van Kulhawy & Mayne (1990) kan nu toegepast worden. Omdat we nu het grondtype kennen voor elke CPT-diepte, kunnen we ervoor kiezen om de correlatie enkel toe te passen op zand (``apply_for_soiltypes=['SAND',]``)

In [ ]:
cpt.apply_correlation('Friction angle Kulhawy and Mayne (1990)', outputs={'Phi [deg]': 'Friction angle Kulhawy [deg]'}, apply_for_soiltypes=['SAND',])

We kunnen de resultaten van de berekening weergeven.

In [ ]:
cpt.data.head()

De eerste rij bevat een $ \infty $, we kunnen Pandas zeggen om die als NaN te behandelen.

In [ ]:
pd.set_option('mode.use_inf_as_na', True)

## Plotten van statigrafie en resultaten van grondonderzoek

De stratigrafie tonen naast de resultaten van grondonderzoek is zeer interessant om te zien of de meetresultaten de gelaagdheid bevestigen en wat de mogelijke intra-laag variabiliteit is. In sommige lagen zullen grondparameters weinig variëren, in andere zullen omwille van geologische redenen, de variaties groter zijn.

Om de stratigrafie en de meetresultaten weer te geven kunnen we de klasse ``LogPlot`` gebruiken.

Vaak hebben we naast gecorreleerde waarden uit CPT ook laboproeven voor bvb de wrijvingshoek (CD triaxiaalproef) verwerken. Hier zullen we ervan uitgaan dat we twee CD proeven hebben, op 6m en 12m diepte met een wrijvingshoek van 43° en 38° respectievelijk. We maken een Python ``list`` met de dieptes en de gemeten waarden.

In [ ]:
friction_angle_depths_cd = [6, 12]
friction_angles_cd = [43, 38]

We kunnen nu een ``LogPlot`` maken die alle meetresultaten en de gelaagdheid weergeeft. De waarden in de kolom ``Soil type`` van ``gelaagdheid`` moeten <i>gemapt</i> worden naar een kleur. We zullen zand geel kleuren, klei bruin en zand/klei mengsels oranje.

De plotting routine hieronder wordt verder toegelicht met commentaar:

In [ ]:
# Maak de LogPlot aan door het SoilProfile op te geven, het aantal panels en de mapping voor grondtypes naar kleuren
qc_phi_plot = LogPlot(soilprofile=gelaagdheid, no_panels=2, fillcolordict={'SAND': 'yellow', 'CLAY': 'brown', 'SAND/CLAY': 'orange'})
# Voeg een trace toe voor conusweerstand in het linkse paneel
qc_phi_plot.add_trace(x=cpt.data['qc [MPa]'], z=cpt.data['z [m]'], name='qc', showlegend=False, panel_no=1)
# Voeg een trace toe voor wrijvingshoek uit de CPT in het rechtse paneel
qc_phi_plot.add_trace(x=cpt.data['Friction angle Kulhawy [deg]'], z=cpt.data['z [m]'], name='from CPT', showlegend=True, panel_no=2)
# Voeg een trace toe voor CD proeven. Dit zijn puntmetingen dus er moeten markers gebruikt worden
qc_phi_plot.add_trace(x=friction_angles_cd, z=friction_angle_depths_cd, name='CD', showlegend=True, panel_no=2, mode='markers')
# Pas de X-as van het linkse panel aan
qc_phi_plot.set_xaxis(title=r'$ q_c \ \text{[MPa]} $', panel_no=1, range=(0, 100))
# Pas de X-as van het rechtse panel aan
qc_phi_plot.set_xaxis(title=r'$ \varphi^{\prime} \ \text{[°]} $', panel_no=2, range=(20, 50))
# Pas de Z-as aan
qc_phi_plot.set_zaxis(title=r'$ z \ \text{[m]} $')
# Geef de grafiek weer
qc_phi_plot.show()

We kunnen met ```groundhog``` ook een automatische selectie van grondparameters uitvoeren. Op basis van bvb de CPT waarden in elke laag zal ``groundhog`` een gemiddelde selecteren en die dan toevoegen aan ons stratigrafisch profiel (``gelaagdheid``).

Eerst zullen we een kopie van de CPT data waarbij we die rijen verwijderen met een NaN waarde voor de wrijvingshoek. Dit kan met Pandas' syntax.

In [ ]:
wrijvingshoek_data = deepcopy(cpt.data.dropna(subset=['Friction angle Kulhawy [deg]',]))
wrijvingshoek_data.head()

De automatische selectie kan worden doorgevoerd met de methode ``.selection_soilparameter``. We kunnen constant of linear varierende waarden kiezen. We geven de naam op van de parameter die in onze ``gelaagdheid`` moet terechtkomen en de waarden waarop de selectie zich kan baseren.

De selectie zelf kijkt naar alle waarden van de wrijvingshoek in een specifieke laag en berekent het gemiddelde voor die laag en kent dat vervolgens toe aan de juiste rij in ``gelaagdheid``.

In [ ]:
cpt.layerdata.selection_soilparameter(
    parameter='Friction angle [deg]',                 # Naam van de parameter in ons SoilProfile gelaagdheid
    depths=wrijvingshoek_data['z [m]'],                         # Dieptes voor de automatische selectie
    values=wrijvingshoek_data['Friction angle Kulhawy [deg]'],  # Wrijvingshoeken voor de automatische selectie
    rule='mean',                                      # Worden gemiddelde waarden, minima of maxima geselecteerd
    linearvariation=False)                            # Constant waarde of lineare variatie in een laag.

Het resultaat kan weergegeven worden

In [ ]:
qc_phi_plot.add_soilparameter_trace('Friction angle [deg]', panel_no=2, legendname='selectie')
qc_phi_plot.show()

Het ``SoilProfile`` ``gelaagdheid`` is ook geupdate met de selectie.

In [ ]:
gelaagdheid

De waarden uit de automatische selectie kunnen ook manueel geupdate worden. Bvb in laag 5 (16.9 tot 20m) kunnen we de geselecteerde waarde verlagen omwille van de grote variabiliteit. In laag 7, de diepste laag, kunnen we de waarde wat verhogen omdat ``groundhog`` de opbouw van conusweerstand heeft meegenomen bij het berekenen van het gemiddelde.

In [ ]:
gelaagdheid.loc[5, 'Friction angle [deg]'] = 38
gelaagdheid.loc[7, 'Friction angle [deg]'] = 45

Het uiteindelijke resultaat kunnen we weergeven. De geselecteerde wrijvingshoek zit in de kolom ``'Friction angle [deg]'`` van het ``SoilProfile`` ``gelaagdheid``. Een kolom uit een ``SoilProfile`` plotten kan heel eenvoudig met de methode ``.add_soilparameter_trace``. Deze methode vereist enkel de naam van de grondmechanische parameter en het <i>panel</i> waarin je die wil plotten.

In [ ]:
phi_selection_plot = LogPlot(soilprofile=gelaagdheid, no_panels=1, fillcolordict={'SAND': 'yellow', 'CLAY': 'brown', 'SAND/CLAY': 'orange'})
phi_selection_plot.add_soilparameter_trace('Friction angle [deg]', panel_no=1)
phi_selection_plot.add_trace(x=cpt.data['Friction angle Kulhawy [deg]'], z=cpt.data['z [m]'], name='Phi', showlegend=False, panel_no=1)
phi_selection_plot.set_xaxis(title=r'$ \varphi^{\prime} \ \text{[-]} $', panel_no=1, range=(20, 50))
phi_selection_plot.set_zaxis(title=r'$ z \ \text{[m]} $')
phi_selection_plot.show()

## Interactieve selectie van grondparameters

``grounghog`` laat ook toe om grondparameters manueel te kiezen. Dit gebeurt met een ``LogPlotMatplotlib`` object, een plot van gelaagdheid en parameters met de bibliotheek Matplotlib. Deze bibliotheek laat toe om te klikken op de grafiek en met de waarden uit deze clicks te werken.

We zullen onze grafieken moeten maken met ``qt``, hiermee worden de grafieken in een apart venster getoond. In dat venster zullen we kunnen klikken.

In [ ]:
%matplotlib qt

We maken eerst een <i>dummy</i> ``SoilProfile`` aan. In de file ``dummyprofile.xlsx`` zit één laag met onbekend grondtype en een totaal eenheidsgewicht van 19kN/m$^3$. We kunnen dit uit Excel inladen met de ``read_excel`` methode.

In [ ]:
dummy_profile = read_excel('Data/dummyprofile.xlsx')
dummy_profile

Vervolgense maken we een grafiek van de conusweerstand en het dummy-profiel. Sluit het venster met deze grafiek NIET! Deze moet blijven openstaan om nadien zelf een gelaagdheid interactief kiezen.

In [ ]:
logplot = LogPlotMatplotlib(
   soilprofile=dummy_profile,
   no_panels=2,
   fillcolordict={'Unknown': 'grey'})
logplot.add_trace(
    x=cpt.data['qc [MPa]'],
    z=cpt.data['z [m]'], name='$ q_c $', panel_no=1, showlegend=False)
logplot.add_trace(
    x=cpt.data['Friction angle Kulhawy [deg]'],
    z=cpt.data['z [m]'], name='CPT', panel_no=2, showlegend=True)
logplot.add_trace(
    x=friction_angles_cd,
    z=friction_angle_depths_cd, name='CD', panel_no=2, showlegend=True, line=False)
logplot.set_xaxis_title(title='$ q_c $ [MPa]', panel_no=1, size=15)
logplot.set_xaxis_title(title=r'$ \varphi^{\prime} $ [°]', panel_no=2, size=15)
logplot.set_zaxis_title(title='$ z $ [m]')
logplot.set_xaxis_range(min_value=-10, max_value=100, panel_no=1)
logplot.set_xaxis_range(min_value=20, max_value=50, panel_no=2)
logplot.set_zaxis_range(min_depth=0, max_depth=30)
logplot.show()

De selectie van gelaagdheid gebeurt door de methode ``seleect_layering`` uit te voeren. Eens we dit commando uitvoeren kunnen we beginnen met klikken op laagovergangen in de grafiek. Als we klikken op een waarde lager dan de ``stop_threshold`` stopt de selectieroutine (anders komen we in een oneindige lus terecht). Daarom dat we onze $ q_c $ hebben laten starten bij -10MPa.

Sluit het venster met de figuur nog NIET! We hebben die nog nodig voor de parameterselectie.

In [ ]:
logplot.select_layering(panel_no=1, stop_threshold=0)

We kunnen de gekozen laagovergangen weergeven.

In [ ]:
dummy_profile

We hebben dezelfde laagovergangen geselecteerd als ervoor. Dus we kunnen ook de eerder gedefinieerde grondtypes en totale eenheidsgewichten toekennen. Als je meer of minder lagen hebt, zul je ook de lijsten met grondtypes en eenheidsgewichten moeten aanpassen.

In [ ]:
dummy_profile['Soil type'] = ['SAND', 'SAND', 'CLAY', 'SAND', 'CLAY', 'SAND', 'SAND/CLAY', 'SAND']
dummy_profile['Total unit weight [kN/m3]'] = [19, 20, 18, 20, 18, 19, 19, 20]
dummy_profile

Op ``SoilProfile`` objecten kun je ook meteen de totale en effectieve spanning berekenen door de eenheidsgewichten te integreren. Dit gebeurt met de methode ``.calculate_overburden``.

In [ ]:
dummy_profile.calculate_overburden(waterlevel=0)
dummy_profile

De interactieve selectie van wrijvingshoeken kan nu ook gebeuren door in de grafiek te klikken. We klikken in het rechtse <i>panel</i> en houden daarbij rekening met zowel de wrijvingshoek uit CPT en de directe metingen. Omdat de metingen uit CD triaxiaalproeven steeds lager uitvallen dan de correlatie, stellen we onze selectie bij naar lagere waarden.

De methode ``select_constant`` laat ons toe om 1 waarde te selecteren in elke laag. We klikken in elk laag van boven naar onder. Als we klikken op een waarde lager dan de ``nan_tolerance`` wordt er een NaN opgenomen in ons ``SoilProfile``. Dit is bvb nodig voor de kleilagen waarbij de correlatie voor zand zeker niet van toepassing is.

Van zodra het commando in uitvoering is, kun je klikken in het venster met de grafiek.

In [ ]:
logplot.select_constant(panel_no=2, parametername='Friction angle sand', units='deg', nan_tolerance=25)

Het resultaat van deze selectie kunnen we weergeven.

In [ ]:
dummy_profile

In [ ]:
phi_manualselection_plot = LogPlot(soilprofile=dummy_profile, no_panels=1, fillcolordict={'SAND': 'yellow', 'CLAY': 'brown', 'SAND/CLAY': 'orange'})
phi_manualselection_plot.add_soilparameter_trace('Friction angle sand [deg]', panel_no=1)
phi_manualselection_plot.add_trace(x=cpt.data['Friction angle Kulhawy [deg]'], z=cpt.data['z [m]'], name='Phi', showlegend=False, panel_no=1)
phi_manualselection_plot.set_xaxis(title=r'$ \varphi^{\prime} \ \text{[-]} $', panel_no=1, range=(20, 50))
phi_manualselection_plot.set_zaxis(title=r'$ z \ \text{[m]} $')
phi_manualselection_plot.show()